In [ ]:
import histogram as bh
import numpy as np

### Testing setup

This is just a simple 1D and 2D dataset to use for performance runs. The testing setup is the same as "MBP" in [this post](https://iscinumpy.gitlab.io/post/histogram-speeds-in-python/).

In [ ]:
bins=(100, 100)
ranges=((-1,1),(-1,1))
bins = np.asarray(bins).astype(np.int64)
ranges = np.asarray(ranges).astype(np.float64)
    
edges = (np.linspace(*ranges[0,:], bins[0]+1),
         np.linspace(*ranges[1,:], bins[1]+1))

In [ ]:
np.random.seed(42)
vals = np.random.normal(size=[2, 1_000_000]).astype(np.float32)
vals1d = np.random.normal(size=[10_000_000]).astype(np.float32)

#### Traditional 1D Numpy Histogram

This is reasonably optimized; it should provide good perforance.

In [ ]:
%%timeit
h, _ = np.histogram(vals1d, bins=bins[0], range=ranges[0])

In [ ]:
h, e = np.histogram(vals1d, bins=bins[0], range=ranges[0])
h

#### Boost histogram: General vector

This can take any axis type

In [ ]:
%%timeit
hist = bh.hist.any_int([bh.axis.regular(bins[0], *ranges[0])])
hist.fill(vals1d)

In [ ]:
hist = bh.hist.regular_int([bh.axis.regular(bins[0], *ranges[0])])
hist.fill(vals1d)
np.array([hist.at(i) for i in range(100)])

#### Boost histogram in 1D: Axis vector

In [ ]:
%%timeit
hist = bh.hist.regular_int([bh.axis.regular(bins[0], *ranges[0])])
hist.fill(vals1d)

In [ ]:
hist = bh.hist.regular_int([bh.axis.regular(bins[0], *ranges[0])])
hist.fill(vals1d)
np.array([hist.at(i) for i in range(100)])

#### Boost histogram in 1D: Dense Tuple

In [ ]:
%%timeit
hist = bh.hist.regular_int_1d((bh.axis.regular(bins[0], *ranges[0]),))
hist.fill(vals1d)

In [ ]:
hist = bh.hist.regular_int_1d((bh.axis.regular(bins[0], *ranges[0]),))
hist.fill(vals1d)
np.array([hist.at(i) for i in range(100)])

---

#### Traditional 2D Numpy histogram

Not as well optimized for regular filling.

In [ ]:
%%timeit
H, *ledges = np.histogram2d(*vals, bins=bins, range=ranges)

In [ ]:
np.histogram2d(*vals, bins=bins, range=ranges)[0][0,0]

#### Boost histogram in 2D: Axis vector

In [ ]:
%%timeit
hist = bh.hist.regular_int([bh.axis.regular(bins[0], *ranges[0]),
                            bh.axis.regular(bins[1], *ranges[1])])
hist.fill(vals)

In [ ]:
hist = bh.hist.regular_int([bh.axis.regular(bins[0], *ranges[0]),
                            bh.axis.regular(bins[1], *ranges[1])])
hist.fill(vals)
hist.at(0,0)

#### Boost histogram in 2D: Axis Tuple

In [ ]:
%%timeit
hist = bh.hist.regular_int_2d((bh.axis.regular(bins[0], *ranges[0]),
                               bh.axis.regular(bins[1], *ranges[1])))
hist.fill(vals)

In [ ]:
hist = bh.hist.regular_int_2d((bh.axis.regular(bins[0], *ranges[0]),
                               bh.axis.regular(bins[1], *ranges[1])))
hist.fill(vals)
hist.at(0,0)